In [22]:
import cv2
import matplotlib.pyplot as plt
import os
import json
import logging
from matplotlib.patches import Rectangle
import numpy as np
import torch
import torchvision.transforms as transforms

In [2]:
from PIL import Image

# Load an image using Pillow
img = Image.open('/home/nele_pauline_suffo/ProcessedData/yolo_gaze_input/train/gaze/quantex_at_home_id262691_2022_03_19_01_019710_face_0.jpg')

# Get the width and height
width, height = img.size

print(f"Width: {width}, Height: {height}")

Width: 438, Height: 233


In [23]:
model = torch.load("/home/nele_pauline_suffo/models/HR18-WFLW.pth")

/tmp/ipykernel_1590653/3086292420.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("/home/nele_pauline_suffo/models/HR18-WFLW.pth")


In [24]:
# run model on an image in inference mode
def run_model(image):
    # set model to evaluation mode
    model.eval()

    # transform image
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])
    image = transform(image)
    image = image.unsqueeze(0)

    # run model
    with torch.no_grad():
        output = model(image)

    return output

In [26]:
image = cv2.imread("/home/nele_pauline_suffo/ProcessedData/quantex_gaze_input/quantex_at_home_id255237_2022_05_08_01_004620_face_0.jpg")
output = run_model(image)
print(output)

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

## Analyze  Face Size

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import logging
import cv2

# Step 1: Parse the Text File
def parse_face_data(file_path, image_dir):
    face_data = []
    skipped_images = 0
    
    with open(file_path, 'r') as file:
        for line in file:
            try:
                parts = line.strip().split()
                image_file = parts[0]
                video_name = image_file[:-11]  # Remove frame number and extension
                image_file_path = os.path.join(image_dir, video_name, image_file)
                
                # Skip if image doesn't exist
                if not os.path.exists(image_file_path):
                    logging.warning(f"Image not found: {image_file_path}")
                    skipped_images += 1
                    continue
                
                bounding_boxes = parts[1:]
                for box in bounding_boxes:
                    try:
                        x, y, width, height, _ = map(float, box.split(','))
                        area = calculate_normalized_area(image_file_path, width, height)
                        face_data.append({
                            "image": image_file,
                            "image_file_path": image_file_path,
                            "x": x, "y": y,
                            "width": width,
                            "height": height,
                            "area": area
                        })
                    except ValueError as e:
                        logging.warning(f"Invalid bbox format in {image_file}: {e}")
                        continue
                        
            except Exception as e:
                logging.error(f"Error processing line: {line.strip()}\n{str(e)}")
                continue
    
    logging.info(f"Processed {len(face_data)} faces, skipped {skipped_images} images")
    return face_data

def calculate_normalized_area(image_path, width, height):
    """
    Calculate the normalized area of the face bounding box relative to the image size.

    Parameters:
        image_path (str): Path to the image file.
        width (float): Width of the bounding box.
        height (float): Height of the bounding box.
    Returns:
        float: Normalized area of the bounding box.
    """
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image not found at path: {image_path}")

    # Get image dimensions
    image_height, image_width = image.shape[:2]

    # Calculate bounding box area
    area = width * height

    # Normalize the area
    normalized_area = area / (image_width * image_height)

    return normalized_area

# Step 2: Define Thresholds and Categorize
def categorize_faces(face_data, small_threshold=135196, medium_threshold=808064):
    for face in face_data:
        if face['area'] < small_threshold:
            face['category'] = 'small'
        elif small_threshold <= face['area'] < medium_threshold:
            face['category'] = 'medium'
        else:
            face['category'] = 'large'
    return face_data

# Step 3: Analyze and Visualize
def analyze_distribution(face_data):
    df = pd.DataFrame(face_data)
    distribution = df['category'].value_counts(normalize=True) * 100
    print("Face Size Distribution (Percentage):")
    print(distribution)

    # Plot histogram with improved visualization
    plt.figure(figsize=(12, 6))
    
    # Plot histogram with more bins
    plt.hist(df['area'], bins=50, color="skyblue", edgecolor="black", alpha=0.7)
    
    # Add thresholds
    plt.axvline(1024, color="red", linestyle="--", linewidth=2, label="Small Threshold (1024px²)")
    plt.axvline(9216, color="green", linestyle="--", linewidth=2, label="Medium Threshold (9216px²)")
    
    # Add mean and median
    plt.axvline(df['area'].mean(), color="purple", linestyle="-.", linewidth=2, label=f"Mean ({df['area'].mean():.0f}px²)")
    plt.axvline(df['area'].median(), color="orange", linestyle="-.", linewidth=2, label=f"Median ({df['area'].median():.0f}px²)")
    
    # Customize axes
    plt.xscale('log')  # Logarithmic scale for better distribution visibility
    plt.grid(True, alpha=0.3)
    plt.xlabel("Face Area (pixels²)", fontsize=12)
    plt.ylabel("Frequency", fontsize=12)
    plt.title("Face Size Distribution (Log Scale)", fontsize=14)
    
    # Improve legend
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    
    plt.show()

    return df

# Step 4: Split Data into Categories
def split_data_by_category(df):
    small_faces = df[df['category'] == 'small']
    medium_faces = df[df['category'] == 'medium']
    large_faces = df[df['category'] == 'large']
    return small_faces, medium_faces, large_faces

In [ ]:
# Main Script
file_path = '/home/nele_pauline_suffo/ProcessedData/mtcnn/face_labels.txt'
image_dir = '/home/nele_pauline_suffo/ProcessedData/quantex_annotated_rawframes'
face_data = parse_face_data(file_path, image_dir)
categorized_faces = categorize_faces(face_data)
df = pd.DataFrame(face_data)
#df = analyze_distribution(categorized_faces)
df.to_pickle('/home/nele_pauline_suffo/ProcessedData/mtcnn/face_data.pkl')
#small_faces, medium_faces, large_faces = split_data_by_category(df)

# Output Results
#print(f"Number of Small Faces: {len(small_faces)}")
#print(f"Number of Medium Faces: {len(medium_faces)}")
#print(f"Number of Large Faces: {len(large_faces)}")
#print(df.head(10))

In [2]:
pip install supervision

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 182.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 117.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gluoncv 0.10.5.post0 requires yacs, which is not installed.
mmdet 3.3.0 requires pycocotools, which is not installed.
ultralytics 8.2.51 requires numpy<2.0.0,>=1.23.0, but you have numpy 1.22.4 which is incompatible.
ultralytics 8.2.51 requires torchvision>=0.9.0, but you have torchvision 0.7.0+cpu which is incompatible.

[notice] A new release of 

In [3]:
import os
import cv2
import logging
from pathlib import Path
from typing import List, Tuple
from ultralytics import YOLO
from supervision import Detections  # Add this import
import numpy as np

def load_model(model_path: Path) -> YOLO:
    """Load YOLO model from path"""
    try:
        model = YOLO(str(model_path))
        logging.info(f"Model loaded successfully from {model_path}")
        return model
    except Exception as e:
        logging.error(f"Failed to load model: {e}")
        raise


def process_image(model: YOLO, image_path: Path) -> Tuple[np.ndarray, Detections]:
    """Process image with YOLO model"""
    image = cv2.imread(str(image_path))
    if image is None:
        raise ValueError(f"Failed to load image from {image_path}")
    
    output = model.predict(image, save=False)
    results = Detections.from_ultralytics(output[0])
    return image, results


def draw_detections(image: np.ndarray, results: Detections) -> np.ndarray:
    """Draw bounding boxes and labels on image"""
    annotated_image = image.copy()
    for bbox, conf in zip(results.xyxy, results.confidence):
        x1, y1, x2, y2 = map(int, bbox)
        cv2.rectangle(annotated_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f"Face: {conf:.2f}"
        cv2.putText(annotated_image, label, (x1, y1 - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return annotated_image


def parse_annotations(label_path: Path) -> List[Tuple[int, float, float, float, float]]:
    """Parse annotations from YOLO label file"""
    annotations = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 5:  # Class, x_center, y_center, width, height
                annotations.append(tuple(map(float, parts)))
    return annotations


def calculate_iou(box1: np.ndarray, box2: np.ndarray) -> float:
    """Calculate Intersection over Union (IoU) between two boxes"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - intersection

    return intersection / union if union > 0 else 0

In [4]:
model = load_model("/home/nele_pauline_suffo/models/yolov11_face_detection.pt")
labels_dir = Path("/home/nele_pauline_suffo/ProcessedData/yolo_face_input/labels/test")
image_path = "/home/nele_pauline_suffo/ProcessedData/yolo_face_input/images/test/quantex_at_home_id260275_2022_04_16_01_014520.jpg"
process_image(image_path)
label_path = labels_dir / (image_path.stem + ".txt")
print("Label Path:", label_path)
ground_truths = parse_annotations(label_path)  # Parse annotations
print("Ground Truths:", ground_truths)

image, results = process_image(model, image_path)
predicted_boxes = results.xyxy
print("Predicted Boxes:", predicted_boxes)

if not ground_truths:  # No ground truth annotations
    if len(predicted_boxes) > 0:  # False positive
        print("False Positive")
    else:  # True negative
        print("True Negative")
    continue

# Evaluate predictions against ground truths
matched_predictions = set()
matched_ground_truths = set()

for pred_idx, pred_box in enumerate(predicted_boxes):
    pred_box = pred_box[:4].astype(float)  # Extract bounding box
    pred_x1, pred_y1, pred_x2, pred_y2 = pred_box

    for gt_idx, gt in enumerate(ground_truths):
        _, x_center, y_center, width, height = gt
        gt_x1 = x_center - width / 2
        gt_y1 = y_center - height / 2
        gt_x2 = x_center + width / 2
        gt_y2 = y_center + height / 2

        iou = calculate_iou(pred_box, [gt_x1, gt_y1, gt_x2, gt_y2])
        if iou >= iou_threshold:
            matched_predictions.add(pred_idx)
            matched_ground_truths.add(gt_idx)
# True Positives
if matched_predictions and matched_ground_truths:
    print("True Positives")

# False Positives: Predictions not matched to any ground truth
unmatched_predictions = set(range(len(predicted_boxes))) - matched_predictions
if unmatched_predictions:
    print("False Positives")

# False Negatives: Ground truths not matched to any predictions
unmatched_ground_truths = set(range(len(ground_truths))) - matched_ground_truths
if unmatched_ground_truths:
    print("False Negatives")

TypeError: process_image() missing 1 required positional argument: 'image_path'

In [1]:
import os

def rename_files(folder_path):
    """
    Renames files in a folder from 'frame_XXXXXX' format to 'quantex_at_home_id257511_2021_07_02_XXXXXX.jpg'.

    Args:
        folder_path (str): The path to the folder containing the files to rename.
    """
    try:
        # Ensure the folder exists
        if not os.path.exists(folder_path):
            print(f"Error: The folder '{folder_path}' does not exist.")
            return

        # List all files in the folder
        files = os.listdir(folder_path)

        for file_name in files:
            # Check if the file matches the pattern 'frame_XXXXXX'
            if file_name.startswith("frame_") and file_name.endswith(".PNG"):
                # Extract the numeric part from the file name
                number_part = file_name.replace("frame_", "").replace(".PNG", "")

                # Construct the new file name
                new_name = f"quantex_at_home_id257511_2021_07_12_02_{number_part}.jpg"

                # Get full paths
                old_path = os.path.join(folder_path, file_name)
                new_path = os.path.join(folder_path, new_name)

                # Rename the file
                os.rename(old_path, new_path)
                print(f"Renamed: {file_name} -> {new_name}")

        print("File renaming completed.")

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
folder_path = "/home/nele_pauline_suffo/ProcessedData/quantex_at_home_id257511_2021_07_12_02_rawframes"  # Replace with your folder path
rename_files(folder_path)

Renamed: frame_000000.PNG -> quantex_at_home_id257511_2021_07_12_02_000000.jpg
Renamed: frame_000030.PNG -> quantex_at_home_id257511_2021_07_12_02_000030.jpg
Renamed: frame_000060.PNG -> quantex_at_home_id257511_2021_07_12_02_000060.jpg
Renamed: frame_000090.PNG -> quantex_at_home_id257511_2021_07_12_02_000090.jpg
Renamed: frame_000120.PNG -> quantex_at_home_id257511_2021_07_12_02_000120.jpg
Renamed: frame_000150.PNG -> quantex_at_home_id257511_2021_07_12_02_000150.jpg
Renamed: frame_000180.PNG -> quantex_at_home_id257511_2021_07_12_02_000180.jpg
Renamed: frame_000210.PNG -> quantex_at_home_id257511_2021_07_12_02_000210.jpg
Renamed: frame_000240.PNG -> quantex_at_home_id257511_2021_07_12_02_000240.jpg
Renamed: frame_000270.PNG -> quantex_at_home_id257511_2021_07_12_02_000270.jpg
Renamed: frame_000300.PNG -> quantex_at_home_id257511_2021_07_12_02_000300.jpg
Renamed: frame_000330.PNG -> quantex_at_home_id257511_2021_07_12_02_000330.jpg
Renamed: frame_000360.PNG -> quantex_at_home_id25751